In [68]:
import pandas as pd
import numpy as np
import scipy.stats as st

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание датасетов:

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [69]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [70]:
df_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [71]:
df_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [72]:
df_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


У нас есть датасеты с данными по 3 регионам, в каждом есть фичи: f0, f1, f2. product - наш таргет. id - неважный параметр, он не влияет на данные. Удалим его.

In [73]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

In [74]:
df_2.sample(1)

,f0,f1,f2,product
24232,-1.063985,0.740587,10.281953,138.679522


Для наглядности, хоть мы и не знаем что у нас значат фичи можно посмотреть их линейную корреляцию, в т.ч. с целевым признаком:

In [75]:
df_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [76]:
df_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [77]:
df_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Мы видим, что во всех трех регионах f2 имеет корреляцию, в случае со вторым регионом корреляция достаточно высокая

## Обучение и проверка модели

In [78]:
def linear_regression(df):
    # разбиение на таргет и фичи
    features = df.drop(["product"], axis=1)
    target = df["product"]
    
    # разделение на тренировочные и валидационные выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, 
        target, 
        test_size=0.25, 
        random_state=13,
    )
    
    # создаем объект MinMaxScaler для масштабирования признаков, по дефолту диапазон default=(0, 1)
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    # создадим модель
    model = LinearRegression()
    
    # обучим на тренировочных данных
    model.fit(features_train, target_train)
    
    # применим обученную модель на валидационных данных
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    
    # посчитаем rmse - корень из среднеквадратичной ошибки
    rmse = (mean_squared_error(predictions, target_valid))**0.5
    # из доки - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    # squaredbool, default=True
    # If True returns MSE value, if False returns RMSE value.

    # average_product = sum(predictions) / len(predictions)
    average_product = predictions.mean()
    
    print(f"RMSE: {rmse:.2f}")
    print(f"Среднее для product: {average_product:.2f}")
    
    return (predictions, target_valid.reset_index(drop=True), rmse)

Для каждого региона посчитаем модель

In [79]:
predictions_0, target_valid_0, rmse_0 = linear_regression(df_0)

RMSE: 37.69
Среднее для product: 92.59


In [80]:
predictions_1, target_valid_1, rmse_1 = linear_regression(df_1)

RMSE: 0.89
Среднее для product: 69.03


In [81]:
predictions_2, target_valid_2, rmse_2 = linear_regression(df_2)

RMSE: 40.02
Среднее для product: 94.96


- **RMSE** - квадратный корень в среднем квадратном различии между оценкой и фактическим значением переменной/функции. 
- В нашем случае это число, которое показывает на сколько в среднем отличается наше предсказание от фактического.
- Мы видим большую ошибку для 1 и 3 регионов и весьма маленькую для 2.

## Подготовка к расчёту прибыли

In [82]:
# Бюджет на разработку скважин в регионе — 10 млрд рублей
budget = 10_000_000_000
# Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
one_barr_income = 450000
# При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
points = 500
best_points = 200
# После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. 
# Среди них выбирают регион с наибольшей средней прибылью.
risk = 0.025

In [83]:
payback_threshold = round(budget/(best_points*one_barr_income), 2)
print(f"Порог, выше которого наша деятельность по разработке месторождения окупится: {payback_threshold} тыс. баррелей")

Порог, выше которого наша деятельность по разработке месторождения окупится: 111.11 тыс. баррелей


## Расчёт прибыли и рисков 

In [84]:
state = np.random.RandomState(13)

Подготовим функцию подсчета профита

In [85]:
def prof(predictions, target):
    # считаем профит, сортируем предсказания по убыванию
    top_preds = predictions.sort_values(ascending=False)
    # берем первые 200(по условию задачи)
    top_target = target[top_preds.index][:200]
    # вычисляем предполагаемый доход с топ200 месторождений
    revenue = top_target.sum() * one_barr_income
    # возвращаем разницу как предполагаемый доход за вычетом вложенного бюджета
    return revenue - budget

Напишем функцию с применением техники бутстрапа, когда мы 1000 раз разделим одну и ту же выборку случайным образом, возьмем таким образом наши предсказания и посчитаем профит по формуле выше. 

In [86]:
def get_confidence_interval_risks(predictions, target):
    revenue = []
    for _ in range(1000):
        target_sample = target.sample(500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        revenue.append(prof(predictions_sample, target_sample))
        
    lower = int(np.percentile(revenue, 2.5))
    higher = int(np.percentile(revenue, 97.5))
    mean_revenue = int(sum(revenue) / len(revenue))
    risk = st.percentileofscore(revenue, 0)

    print(f"Средняя выручка {mean_revenue} млн. руб")
    print(f"Доверительный интервал '{round(lower, 2)}' - '{round(higher, 2)}' млн. руб")
    print(f"Выборочное стандартное отклонение прибыли {round(pd.Series(revenue).sem(), 1)} млн. руб")
    print(f"Риск потерь {risk}")
    return ((lower, higher), mean_revenue, risk)

Для первого региона:

In [87]:
interval, mean_revenue, risk = get_confidence_interval_risks(predictions_0, target_valid_0)

Средняя выручка 473071601 млн. руб
Доверительный интервал '-59395811' - '985562309' млн. руб
Выборочное стандартное отклонение прибыли 8822446.5 млн. руб
Риск потерь 4.8


Для второго региона:

In [88]:
interval, mean_revenue, risk = get_confidence_interval_risks(predictions_1, target_valid_1)

Средняя выручка 523433711 млн. руб
Доверительный интервал '115994263' - '945261099' млн. руб
Выборочное стандартное отклонение прибыли 6787444.3 млн. руб
Риск потерь 0.7


Для третьего региона:

In [89]:
interval, mean_revenue, risk = get_confidence_interval_risks(predictions_2, target_valid_2)

Средняя выручка 398861704 млн. руб
Доверительный интервал '-143217017' - '936244185' млн. руб
Выборочное стандартное отклонение прибыли 8784360.9 млн. руб
Риск потерь 8.5


Вывод:
- Провели визуальную оценку датасета, обратили внимание на корреляцию фичей.
- Построили линейную модель, для предсказания объемов нефти в скважинах для трех регионов.
- Провели процедуру bootstrap и нашли 95% доверительный интервал.
- По результату видим что во втором регионе доверительный интервал у нас не имеет отрицательных значений.
- Потернциально нам интересно будет вести разработку в регионе 2, так как потенциальные риски почти равны нулю.